# ***5.Visualz-plotly-sairauspaaryhmat-2003-2023***
merge/merge1&_avoin-data_suomen_työkyvyttömyyseläkkeensaajat_2003-2023_merge.csv

In [384]:
import pandas as pd
import plotly.express as px
import plotly.io as pio

In [385]:
df = pd.read_csv("merge/merge1&_avoin-data_suomen_työkyvyttömyyseläkkeensaajat_2003-2023_merge.csv")
df.sairauspaaryhma.unique()

array(['I00-I99 Verenkiertoelinten sairaudet',
       'M00-M99 Tuki- ja liikuntael. sekä sidekudoksen sair.',
       'F00-F99 Mielenterveyden ja käyttäytymisen häiriöt', 'Yhteensä',
       'Tieto puuttuu',
       'Q00-Q99 Synnynn. epämuod. ja kromosomipoikkeavuudet',
       'L00-L99 Ihon ja ihonalaiskudoksen sairaudet',
       'J00-J99 Hengityselinten sairaudet',
       'S00-T99 Vammat, myrkytykset ja muut ulk. syiden seur.',
       'G00-G99 Hermoston sairaudet', 'C00-D49 Kasvaimet',
       'E00-E99 Umpieritys-, ravitsemus- ja aineenvaihduntasair.',
       'K00-K99 Ruoansulatuselinten sairaudet',
       'N00-N99 Virtsa- ja sukupuolielinten sairaudet',
       'H00-H59 Silmän ja sen apuelinten sairaudet',
       'H60-H99 Korvan ja kartiolisäkkeen sairaudet',
       'A00-B99 Tartunta- ja loistaudit',
       'Muut pääryhmät (XV,XVI,XVIII,XX)',
       'D50-D99 Veren ja vertamuodostavien elinten sairaudet'],
      dtype=object)

In [386]:
df = df[(df['sairauspaaryhma'] != 'Yhteensä') & (df['sairauspaaryhma'] != 'Tieto puuttuu')] # & (df['vuosi'] == 2019)]

In [387]:
all_locations = df.groupby('Maakunta').agg(
    saaja_lkm=('saaja_lkm', 'count'),
    saaja_lkm_mies=('sukupuoli', lambda x: (x == 'Mies').sum()),
    saaja_lkm_nainen=('sukupuoli', lambda x: (x == 'Nainen').sum()),
    saaja_lkm_muu=('sukupuoli', lambda x: ((x != 'Mies') & (x != 'Nainen')).sum())
).reset_index()

In [388]:
fig1 = px.bar(
    all_locations, 
    x='Maakunta', 
    y='saaja_lkm',
    color='saaja_lkm',
    labels={'saaja_lkm': 'LKM', 'Maakunta': 'Maakunta'},
)

fig1.update_traces(
    customdata=all_locations[['saaja_lkm','saaja_lkm_mies', 'saaja_lkm_nainen', 'saaja_lkm_muu']],
    hovertemplate='<b>%{x}</b><br>' +
                  '<i>Yhteensä: %{customdata[0]}</i><br>' +
                  '<i>Miehet: %{customdata[1]}</i><br>' +
                  '<i>Naiset: %{customdata[2]}</i><br>' +
                  '<i>Muu: %{customdata[3]}</i><extra></extra>'
)

fig1.update_layout(
    hoverlabel_align = 'right',
    title='<b>2003-2023, Työkyvyttömyyseläkkeensaajat yhteensä maakunnittain</b>'
    )

fig1.show()
pio.write_html(fig1, file='maakunta_sairaspaaryhmat_yhteensa.html', auto_open=False)

In [389]:
grouped = df.groupby(['Maakunta', 'sairauspaaryhma',]).agg(
    saaja_lkm=('saaja_lkm', 'count')
).reset_index()

fig2 = px.bar(
    grouped, 
    y='Maakunta', 
    x='saaja_lkm', 
    color='sairauspaaryhma',
    labels={
        'Maakunta': 'Maakunta', 
        'saaja_lkm': 'LKM', 
        'sairauspaaryhma': 'Sairauspääryhmä'
    },
    barmode='group',
    height=900
)

fig2.update_traces(
    customdata=all_locations[['saaja_lkm','saaja_lkm_mies', 'saaja_lkm_nainen', 'saaja_lkm_muu']],
    hovertemplate='<b>%{y}</b><br>' +
                  '<i>Yhteensä: %{customdata[0]}</i><br>' +
                  '<i>Miehet: %{customdata[1]}</i><br>' +
                  '<i>Naiset: %{customdata[2]}</i><br>' +
                  '<i>Muu: %{customdata[3]}</i><extra></extra>'
)


fig2.update_layout(
    hoverlabel_align = 'right',
    title="<b>2003-2023, Työkyvyttömyyseläkkeensaajat maakunnittain eriteltynä.</b>"
    )

fig2.show()
pio.write_html(fig2, file='maakunta_sairaspaaryhmat_eriteltyna.html', auto_open=False)


In [390]:
uusimaa_df = df[df['Maakunta'] == 'Uusimaa']

grouped_uusimaa = uusimaa_df.groupby(['sairauspaaryhma','vuosi']).agg(
    saaja_lkm=('saaja_lkm', 'count'),
    saaja_lkm_mies=('sukupuoli', lambda x: (x == 'Mies').sum()),
    saaja_lkm_nainen=('sukupuoli', lambda x: (x == 'Nainen').sum()),
    saaja_lkm_muu=('sukupuoli', lambda x: ((x != 'Mies') & (x != 'Nainen')).sum())
).reset_index()

fig3 = px.line(
    grouped_uusimaa, 
    x='vuosi', 
    y='saaja_lkm', 
    color='sairauspaaryhma',
    labels={
        'vuosi': 'Vuosi', 
        'saaja_lkm': 'LKM', 
        'sairauspaaryhma': 'Sairauspääryhmä'
    },
    height=900, 
    category_orders={"vuosi": sorted(uusimaa_df['vuosi'].unique())}
)

fig3.update_traces(
    hovertemplate='<b>%{x}</b><br>'+'<b>%{y}:LKM</b><br>'
)

fig3.update_layout(
    hoverlabel_align='right',
    title="<b>2003-2023, Työkyvyttömyyseläkkeensaajien vuosikehitys, Uudenmaan alueella.</b>"
)

fig3.update_xaxes(tickangle=40, type='category')
fig3.show()
pio.write_html(fig3, file='vuosikehitys_2003-2023_uusimaa.html', auto_open=False)